In [1]:

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
# Referência: https://pandas.pydata.org/pandas-docs/stable/user_guide/sparse.html
from scipy import sparse
import time
# Display the generated image:
# the matplotlib way:
import matplotlib.pyplot as plt
%matplotlib inline
import os
from os import path
from wordcloud import WordCloud
import nltk
from nltk import tokenize
from nltk.corpus import stopwords
nltk.download('all')

/home/emmanuel/.pyenv/versions/3.6.8/envs/cranium/lib/python3.6/site-packages/pandas/compat/__init__.py:120: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)


[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package biocreative_ppi is already up-to-date!
[nltk_data]    | Downloading package brown to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package brown is already up-to-date!
[nltk_data]    | Downloading package brown_tei to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package brown_tei is already up-to-date!
[nltk_data]    | Downloading package cess_cat to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package cess_cat is already up-to-date!
[nl

[nltk_data]    |   Package timit is already up-to-date!
[nltk_data]    | Downloading package toolbox to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package toolbox is already up-to-date!
[nltk_data]    | Downloading package treebank to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package treebank is already up-to-date!
[nltk_data]    | Downloading package twitter_samples to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package twitter_samples is already up-to-date!
[nltk_data]    | Downloading package udhr to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package udhr is already up-to-date!
[nltk_data]    | Downloading package udhr2 to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package udhr2 is already up-to-date!
[nltk_data]    | Downloading package unicode_samples to
[nltk_data]    |     /home/emmanuel/nltk_data...
[nltk_data]    |   Package unicode_samples is already up

True

In [2]:
start_time = time.time()


resenhas_imdb = pd.read_csv('../../dados_imdb/imdb-reviews-pt-br.csv')

'''
# PARA MOSTRAR TODOS
#https://stackoverflow.com/questions/62207066/pandas-does-not-show-the-complete-csv-file
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  
    display(resenhas_imdb)
'''

resenhas_imdb.head()

,id,text_en,text_pt,sentiment
0,1,Once again Mr. Costner has dragged out a movie...,"Mais uma vez, o Sr. Costner arrumou um filme p...",neg
1,2,This is an example of why the majority of acti...,Este é um exemplo do motivo pelo qual a maiori...,neg
2,3,"First of all I hate those moronic rappers, who...","Primeiro de tudo eu odeio esses raps imbecis, ...",neg
3,4,Not even the Beatles could write songs everyon...,Nem mesmo os Beatles puderam escrever músicas ...,neg
4,5,Brass pictures movies is not a fitting word fo...,Filmes de fotos de latão não é uma palavra apr...,neg


In [3]:
resenhas_imdb['classificacao'] = resenhas_imdb['sentiment'].replace(['neg', 'pos'], [0,1])

In [21]:
def treinar_classificador(com_tratamento=True):

    vetorizador = CountVectorizer(lowercase=False, max_features=400)

    tamanho_texto_integral = len(resenhas_imdb['text_pt'])
    
    if com_tratamento == True:

        palavras_irrelevantes = nltk.corpus.stopwords.words("portuguese")
        from string import punctuation
        for pontuacao in punctuation:
            palavras_irrelevantes.append(pontuacao)
            
    else:
        palavras_irrelevantes = []
        
        
    frase_processada = list()

    #tokenizador = tokenize.WhitespaceTokenizer()
    tokenizador = tokenize.WordPunctTokenizer()

    
    #Retirada de acentos
    import unidecode
    
    
    for opiniao in resenhas_imdb.text_pt:
        palavras_texto = tokenizador.tokenize(opiniao)
        
        if com_tratamento == True:
            nova_frase = [unidecode.unidecode(palavra) for palavra in palavras_texto if palavra not in palavras_irrelevantes]
        else:
            nova_frase = [palavra for palavra in palavras_texto if palavra not in palavras_irrelevantes]
            
        frase_processada.append(' '.join(nova_frase))
        
    if com_tratamento == True:
        resenhas_imdb["texto_pt_tratado"] = frase_processada
        texto = 'texto_pt_tratado'
        
    else:
        texto = 'text_pt'

    
    bag_of_words = vetorizador.fit_transform(resenhas_imdb[texto])

    '''
    matriz_esparsa = pd.DataFrame.sparse.from_spmatrix(bag_of_words,
                                                      columns=vetorizador.get_feature_names())
    '''

    treino, teste, classe_treino, classe_teste = train_test_split(bag_of_words,
                                                                  resenhas_imdb.classificacao,
                                                                  random_state=42)

    # https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
    # https://stackoverflow.com/questions/62658215/convergencewarning-lbfgs-failed-to-converge-status-1-stop-total-no-of-iter
    regressao_logistica = LogisticRegression(solver='lbfgs', max_iter=200)
    regressao_logistica.fit(treino, classe_treino)
    acuracia = regressao_logistica.score(teste, classe_teste)

    return acuracia

## Testando acurácia sem tratamento do texto 

In [22]:
# https://www.ic.unicamp.br/~mc102/mc102-1s2019/labs/format.html
'''
Testando acurácia sem tratamento do texto

'''
print('Acurácia %.2f%%' % (treinar_classificador(com_tratamento=False) * 100))

Acurácia 80.84%


In [24]:
resenhas_imdb["text_pt"]

0        Mais uma vez, o Sr. Costner arrumou um filme p...
1        Este é um exemplo do motivo pelo qual a maiori...
2        Primeiro de tudo eu odeio esses raps imbecis, ...
3        Nem mesmo os Beatles puderam escrever músicas ...
4        Filmes de fotos de latão não é uma palavra apr...
                               ...                        
49454    Como a média de votos era muito baixa, e o fat...
49455    O enredo teve algumas reviravoltas infelizes e...
49456    Estou espantado com a forma como este filme e ...
49457    A Christmas Together realmente veio antes do m...
49458    O drama romântico da classe trabalhadora do di...
Name: text_pt, Length: 49459, dtype: object

## Testando acurácia com tratamento do texto 

In [23]:
# https://www.ic.unicamp.br/~mc102/mc102-1s2019/labs/format.html
print('Acurácia %.2f%%' % (treinar_classificador() * 100))

resenhas_imdb["texto_pt_tratado"]

Acurácia 81.06%


0        Mais vez Sr Costner arrumou filme tempo necess...
1        Este exemplo motivo maioria filmes acao mesmos...
2        Primeiro tudo odeio raps imbecis poderiam agir...
3        Nem Beatles puderam escrever musicas todos gos...
4        Filmes fotos latao palavra apropriada verdade ...
                               ...                        
49454    Como media votos baixa fato funcionario locado...
49455    O enredo algumas reviravoltas infelizes inacre...
49456    Estou espantado forma filme maioria outros med...
49457    A Christmas Together realmente veio antes temp...
49458    O drama romantico classe trabalhadora diretor ...
Name: texto_pt_tratado, Length: 49459, dtype: object

In [6]:
tempo_execucao = time.time() - start_time
print('-----Tempo de execução: %s segundos' % tempo_execucao)

-----Tempo de execução: 30.546679735183716 segundos
